In [1]:
# Qui ci metto gli import

import os
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import librosa

import sys
sys.path.append('/home/simone')

import numpy as np

# Aggiungi le funzioni necessarie per il caricamento del modello e l'interpolazione
from VideoMamba.Train_AudioMamba3 import Params
from VideoMamba.DiffWave_simone3 import DiffWave
import pandas as pd
import time

/home/shiva/miniconda3/envs/amamba/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/shiva/miniconda3/envs/amamba/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/shiva/miniconda3/envs/amamba/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [2]:
# Auxiliary functions

def get_random_file(directory_path = '/media/nvme_4tb/simone_data/VoiceBank/clean_testset_wav/'):
    """
    Prende in input una cartella con i file di test, ed estrae il path di un file casualmente
    """
    # Prendo la lista dei file
    files = [f for f in os.listdir(directory_path) if os.path.isfile(os.path.join(directory_path, f))]
    
    # Se la lista non è vuota, prendi un file random
    if files:
        random_file = random.choice(files)
        return os.path.join(directory_path, random_file)
    else:
        return None
    
def get_conditioning_path(original_path):
    """
    Prende il path del file di test, e lo modifica per avere quello del corrispondente file a 24KhZ
    """
    # Sostituisce 'clean_testset_wav' con 'clean_testset_wav_24khz'
    new_path = original_path.replace('clean_testset_wav', 'clean_testset_wav_24khz')
    return new_path    


def snr(pred, target):
    """
    Implementazione NU-Wave per calcolare il Signal-to-Noise Ratio (SNR)
    """
    pred = torch.tensor(pred)
    target = torch.tensor(target)
    return (20 *torch.log10(torch.norm(target, dim=-1) \
                /torch.norm(pred -target, dim =-1).clamp(min =1e-8))).mean()    

class STFTMag(nn.Module):
    """
    Classe presa da NU-Wave per calcolare la Log-Spectral Distance (LSD)
    """
    def __init__(self, nfft=1024, hop=256):
        super().__init__()
        self.nfft = nfft
        self.hop = hop
        self.register_buffer('window', torch.hann_window(nfft), False)

    # x: [B, T] or [T]
    @torch.no_grad()
    def forward(self, x):
        if x.dim() == 3:
            x = x.squeeze(1)
        T = x.shape[-1]
        stft = torch.stft(x,
                          self.nfft,
                          self.hop,
                          window=self.window,
                          return_complex=True)  # Impostato return_complex=True
        mag = torch.abs(stft)  # Calcolo della magnitudine per valori complessi
        return mag

stft = STFTMag()
def lsd(pred, target):
    sp = torch.log10(stft(pred).square().clamp(1e-8))
    st = torch.log10(stft(target).square().clamp(1e-8))
    return (sp - st).square().mean(dim=1).sqrt().mean()    
    

# Funzione di caricamento del modello dal checkpoint
def load_model_from_checkpoint(checkpoint_path, model_class, params, device):
    model = model_class(params).to(device)
    checkpoint = torch.load(checkpoint_path, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])

    # for k, v in checkpoint.items():
    #     if "norm_layer" in k:
    #         checkpoint.pop(k)

    model.eval()  # Imposta il modello in modalità valutazione
    return model

def interpolate_audio_signal(audio_signal, scale_factor = 2):
    """
    Effettua un'interpolazione lineare su un segnale audio PyTorch Tensor per raddoppiarne la lunghezza.
    
    :param audio_signal: Tensor di PyTorch contenente il segnale audio. Dimensioni previste [batch_size, channels, length].
    :param scale_factor: Fattore di scala per la lunghezza del segnale. Es: 2 per raddoppiare la lunghezza.
    :return: Tensor di PyTorch contenente il segnale audio interpolato.
    """
    # Interpolazione lungo l'ultimo asse
    # mode='linear' quando lavori con 3D assume 'linear' lungo l'asse W di [N, C, L]
    # align_corners=False per evitare artefatti agli estremi
    interpolated_signal = F.interpolate(audio_signal, scale_factor=scale_factor, mode='linear', align_corners=False)
    return interpolated_signal

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
params = Params(
        residual_channels=32,
        noise_schedule_params=(1e-6, 0.006, 500),
        unconditional=False,
        n_mels=10,
        residual_layers=11,
        dilation_cycle_length=5000, #150
        device=device
    )    

In [3]:
# Ogni audio ci mette tra i 20 e i 50 secondi per essere generato.
# Quindi, scegliendo N audio il tempo necessario per la generazione è
# compreso tra i 0,33*N e i 0,85*N minuti. Ad esempio, con N=100 abbiamo
# un tempo di generazione in [33; 85] minuti. Ci riduciamo dunque a N=50
# per non appesantire troppo la macchina.

# results_model_2.csv: modello addestrato con la MSE loss
# results_model_1.csv: modello addestrato con la L1 loss

n = 100
checkpoint_path = "/media/nvme_4tb/simone_data/VoiceBank/checkpoints_NEW/checkpoint_epoch_5.pt"
model = load_model_from_checkpoint(checkpoint_path, DiffWave, params, device)
min_steps = 200
max_steps = 500
csv_file = '/home/simone/VideoMamba/results_model_2.csv'

if not os.path.exists(csv_file):
    # Se il file non esiste, crea un nuovo DataFrame vuoto
    df = pd.DataFrame(columns=['Length', 'Num. Steps', 'SNR', 'LSD'])
else:
    # Se il file esiste, carica il DataFrame dal file CSV
    df = pd.read_csv(csv_file)


for step_processing in range(n):
    target_path = get_random_file(directory_path = '/media/nvme_4tb/simone_data/VoiceBank/clean_testset_wav/')
    target_audio, sr = librosa.load(target_path, sr=48000)
    target_audio = torch.from_numpy(target_audio).unsqueeze(0).unsqueeze(0)

    conditioning_path = get_conditioning_path(target_path)
    conditioning_audio, sr = librosa.load(target_path, sr=24000)
    conditioning_audio_tensor = torch.from_numpy(conditioning_audio).unsqueeze(0).unsqueeze(0)  # [1, 1, L]
    conditioned_audio_interpolated = interpolate_audio_signal(conditioning_audio_tensor, scale_factor=2)

    length_signal = 2 * conditioning_audio.shape[-1]
    
    input_audio = torch.randn(2 * conditioning_audio.shape[-1])
    input_audio = input_audio.unsqueeze(0).unsqueeze(0).to(device)

    if input_audio.size(-1) != target_audio.size(-1):
        # Se y ha una lunghezza dispari minore di conditioner, "limiamo" l'ultimo valore di conditioner.
        # Questo avviene perché raddoppiando la dimensione del conditioning, la lunghezza sarà sempre pari, ma magari
        # l'audio originale a 48KhZ era di lunghezza disparo
        input_audio = input_audio[..., :target_audio.size(-1)]

    #####
    num_steps = random.randint(min_steps, max_steps)
    params = Params(
        residual_channels=32,
        noise_schedule_params=(1e-6, 0.006, num_steps),
        unconditional=False,
        n_mels=10,
        residual_layers=11,
        dilation_cycle_length=5000, #150
        device=device
    ) 

    start_time = time.time()
    sampled_audio = model.sample(steps=len(params.noise_schedule), conditioning=conditioning_audio_tensor, audio_length=input_audio.size(-1))
    end_time = time.time()
    sampled_audio_np = sampled_audio.squeeze().cpu().numpy()

    time_generation = end_time - start_time

    snr_value = snr(sampled_audio_np, target_audio)
    lsd_value = lsd(torch.from_numpy(sampled_audio_np).unsqueeze(0).to(target_audio.device), target_audio)

    new_row = pd.DataFrame({'Length': length_signal, 'Num. Steps': num_steps, 'Time Generation': time_generation,'SNR': snr_value, 'LSD': lsd_value}, index=[0])
    df = pd.concat([df, new_row])

    print(f'Step {step_processing} completato')


df.to_csv(csv_file, index=False)

/tmp/ipykernel_2272657/2347849639.py:31: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  target = torch.tensor(target)


Step 0 completato


/tmp/ipykernel_2272657/2347849639.py:31: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  target = torch.tensor(target)


Step 1 completato


/tmp/ipykernel_2272657/2347849639.py:31: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  target = torch.tensor(target)


...

/tmp/ipykernel_2272657/2347849639.py:31: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  target = torch.tensor(target)


Step 97 completato


/tmp/ipykernel_2272657/2347849639.py:31: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  target = torch.tensor(target)


Step 98 completato
Step 99 completato


/tmp/ipykernel_2272657/2347849639.py:31: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  target = torch.tensor(target)


# Analisi dei risultati

In [4]:
# Model 1:
import pandas as pd

# Carica il CSV in un DataFrame (sostituisci 'tuo_file.csv' con il percorso al tuo file)
df = pd.read_csv('/home/simone/VideoMamba/results_model_2.csv')

df['SNR'] = df['SNR'].str.extract(r'tensor\((.*?)\)').astype(float)
df['LSD'] = df['LSD'].str.extract(r'tensor\((.*?)\)').astype(float)

# Calcola la media di ogni colonna
mean_values = df.mean()

# Stampa le medie
print(mean_values)


Length             113721.880000
Num. Steps            358.470000
SNR                     0.366871
LSD                     2.140799
Time Generation        25.977289
dtype: float64
